# MPDP: Visco-elasto-plasticity with isotropic hardening

Associated, rate-dependent plasticity. .

Application of GSM MPDP - Dissipation potential for visco-elasticity included


In [ ]:
%matplotlib widget
from bmcs_matmod.api import GSMMPDP
import matplotlib.pylab as plt
import sympy as sp
import numpy as np
sp.init_printing()

### Helmholtz free energy

In [ ]:
E = sp.Symbol(r'E', real=True, nonnegative=True)
K = sp.Symbol(r'K', real=True)
f_c = sp.Symbol(r'f_\mathrm{c}')
eta = sp.Symbol(r'\eta', real=True, nonnegative=True)

mparams = (E, K, f_c, eta)

# ## External state variables

eps = sp.Symbol(r'\varepsilon', real=True)
eps_a = sp.Matrix([eps])
sig = sp.Symbol(r'\sigma', real=True)
sig_a = sp.Matrix([sig])

# ## Internal state variables

eps_p = sp.Symbol(r'\varepsilon^\mathrm{p}', real=True)
eps_p_a = sp.Matrix([eps_p])
sig_p = sp.Symbol(r'\sigma^\mathrm{p}', real=True)
sig_p_a = sp.Matrix([sig_p])

z = sp.Symbol(r'z', real=True, nonnegative=True)
z_a = sp.Matrix([z])
Z = sp.Symbol(r'Z', real=True, nonnegative=True)
Z_a = sp.Matrix([Z])

# ## Free energy potential
eps_el = eps - eps_p
U_e_ = sp.Rational(1,2) * E * eps_el**2
U_p_ =  sp.Rational(1,2) * K * z**2
F_ = U_e_ + U_p_

# ## Flow potential
f_ = sp.sqrt(sig_p**2) - (f_c + Z)
phi_ext_ = 0

Eps_vars = (eps_p_a, z_a)
Sig_vars = (sig_p_a, Z_a)
Sig_signs =  (-1, 1)


### GSM drivers

In [ ]:
gsm_vep = GSMMPDP(
    name = 'gsm_F_1d_mpdp_vep_lih',
    u_vars = eps_a,
    sig_vars = sig_a,
    m_params = mparams,
    Eps_vars = Eps_vars,
    Sig_vars = Sig_vars,
    Sig_signs = Sig_signs,
    F_expr = F_,
    f_expr = f_,
#    phi_ext_expr = phi_ext_,
)

In [ ]:
# dot_z = gsm_vep.dot_Eps[1, 0]
# pi_mech = -sp.Rational(1,2) * eta * dot_z**2
# gsm_vep.pi_expr = pi_mech
#pi_mech.diff(gsm_vep.dot_Eps.as_explicit())

In [ ]:
gsm_vep.phi_

In [ ]:
gsm_vep.Sig_, gsm_vep.phi_.diff(gsm_vep.Sig.as_explicit()), gsm_vep.Y_, gsm_vep.Phi_

In [ ]:
(gamma_mech, L_, dR_dA_n1), (eps_n, delta_eps, Eps_n, delta_A, delta_t, Ox, Ix), Sig_n1, f_n1, R_n1, dR_dA_n1 = gsm_vep.Sig_f_R_dR_n1
gamma_mech

In [ ]:
L_

In [ ]:
Sig_n1

In [ ]:
R_n1_subs = R_n1.replace(sp.sign, lambda x: 1)
R_n1_subs

In [ ]:
delta_A.T

In [ ]:
dR_dA_n1_subs = dR_dA_n1.replace(sp.sign, lambda x: 1)
dR_dA_n1_subs

In [ ]:
sp.simplify((dR_dA_n1_subs.inv() * R_n1_subs))

In [ ]:
mparams_epd = {E:1, K:1, f_c:1, eta:100}
margs = [mparams_epd[p] for p in mparams]
n_t = 30
n_I = 1
eps_max = 6
t_t = np.linspace(0, 1, n_t)
eps_n_t = np.linspace(0, eps_max, n_t)
Eps_n_Ib = np.zeros((n_I, gsm_vep.n_Eps_explicit), dtype=np.float_)
t_t, eps_ta_nr_coarse, Eps_t, Sig_t_nr_coarse, iter_t, lam_t, (d_t_t, d_eps_ta) = gsm_vep.get_response(eps_n_t, t_t, 100, *margs)

In [ ]:
fig, ax = plt.subplots(1, 1)
ax.plot(eps_ta_nr_coarse, Sig_t_nr_coarse[:,:,0], color='red')

In [ ]:
mparams_epd = {E:1, K:1e-5, f_c:1, eta:1}
margs = [mparams_epd[p] for p in mparams]
eps_n_I = np.array([0, 1.0, 1.0, -1.0, 0, 0], dtype=np.float_)
d_eps_I = np.array([0.1, -0.1, 0.1, -0.1, 2, -2], dtype=np.float_)
n_I = eps_n_I.shape[0]
d_t = 1
Eps_n_Ib = np.zeros((n_I, gsm_vep.n_Eps_explicit), dtype=np.float_)

Eps_n1_Ib, Sig_n1_Ib, lam_I, k_I = gsm_vep.get_state_n1(eps_n_I, d_eps_I, d_t, Eps_n_Ib, 30, *margs )
Eps_n1_Ib, Sig_n1_Ib, lam_I, k_I